In [ ]:
%pip install transformers
%pip install evaluate
%pip install accelerate -U
%pip install torch torchvision
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments,Seq2SeqTrainer, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments
import evaluate
import csv
import random
import torch
from torch.utils.data import TensorDataset
from torch.nn.utils.rnn import pad_sequence
import pandas as pd
import numpy as np
import nltk
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
nltk.download('punkt')
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 42.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 66.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 75.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
%pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=eb6d3e2e0a7cfa0b0ea94c1ddd2d8a3e55c8cce7e8f9781cb19e6dee2ad8452b
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
import pandas as pd
import torch
from transformers import T5TokenizerFast
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader

# Define a custom dataset class
class CustomDataset(Dataset):
    def __init__(self, data, tokenizer, input_max_length=1024, output_max_length=256):
        self.data = data
        self.tokenizer = tokenizer
        self.input_max_length = input_max_length
        self.output_max_length = output_max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        input_text = self.data.iloc[index]['input']
        target_text = self.data.iloc[index]['output']

        # Tokenize input and target text
        input_tokens = self.tokenizer.encode_plus(
            input_text,
            max_length=self.input_max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt',
            return_token_type_ids=False,
            return_attention_mask=True,
            return_overflowing_tokens=False,
            return_special_tokens_mask=False
        )
        target_tokens = self.tokenizer.encode(
            target_text,
            max_length=self.output_max_length,
            truncation=True,
            return_tensors='pt'
        )

        return {
            'input_ids': input_tokens['input_ids'].flatten(),
            'attention_mask': input_tokens['attention_mask'].flatten(),
            'labels': target_tokens.flatten()
        }

# Load CSV data
data_df = pd.read_csv("/content/train_cv_data.csv")  # Change "your_data.csv" to your CSV file path

# Split data into train and evaluation sets
train_df, eval_df = train_test_split(data_df, test_size=0.2, random_state=42)

# Initialize T5 tokenizer
tokenizer = T5TokenizerFast.from_pretrained('t5-small')
tokenizer.add_special_tokens({'eos_token':'[EOS]'})

# Create datasets
train_dataset = CustomDataset(train_df, tokenizer)
eval_dataset = CustomDataset(eval_df, tokenizer)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
eval_loader = DataLoader(eval_dataset, batch_size=16, shuffle=False)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from transformers import T5ForConditionalGeneration, Trainer, TrainingArguments
from datasets import load_metric

# Define the T5 model
model = T5ForConditionalGeneration.from_pretrained("t5-small")
model.resize_token_embeddings(len(tokenizer))

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./drive/MyDrive/Model_SparkLab/t5-small-finetuned",  # output directory
    num_train_epochs=10,  # total number of training epochs
    per_device_train_batch_size=4,  # batch size per device during training
    per_device_eval_batch_size=4,   # batch size for evaluation
    logging_dir="./logs",  # directory for storing logs
    logging_steps=100,  # log every n updates steps
    evaluation_strategy="steps",  # evaluation strategy to use
    eval_steps=200,  # evaluate every n steps
    save_steps=500,  # save checkpoint every n steps
    save_total_limit=2,  # limit the total number of checkpoints
    report_to="tensorboard",  # enable TensorBoard logging
    prediction_loss_only=True,  # only compute the prediction loss
    overwrite_output_dir=True  # overwrite the content of the output directory
)
# Define metric for evaluation
metric = load_metric("rouge")

# Define function for compute metrics during evaluation
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    rouge_output = metric.compute(predictions=predictions, references=labels, use_stemmer=True)
    return {
        "rouge1_precision": rouge_output["rouge1"].precision,
        "rouge1_recall": rouge_output["rouge1"].recall,
        "rouge1_fmeasure": rouge_output["rouge1"].fmeasure,
        "rouge2_precision": rouge_output["rouge2"].precision,
        "rouge2_recall": rouge_output["rouge2"].recall,
        "rouge2_fmeasure": rouge_output["rouge2"].fmeasure,
        "rougeL_precision": rouge_output["rougeL"].precision,
        "rougeL_recall": rouge_output["rougeL"].recall,
        "rougeL_fmeasure": rouge_output["rougeL"].fmeasure,
    }

# Create Trainer
trainer = Trainer(
    model=model,  # the instantiated Transformers model to be trained
    args=training_args,  # training arguments
    train_dataset=train_dataset,  # training dataset
    data_collator=data_collator,
    eval_dataset=eval_dataset,  # evaluation dataset
    compute_metrics=compute_metrics,  # compute metrics function
)

# Train the model
trainer.train()

# Evaluate the model
eval_results = trainer.evaluate()
# print(eval_results)


/usr/local/lib/python3.10/dist-packages/datasets/load.py:756: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=

Step,Training Loss,Validation Loss
200,2.886800,2.547167
400,2.557800,2.352702
600,2.454600,2.264148
800,2.353200,2.214953
1000,2.343000,2.183907
1200,2.282600,2.165350
1400,2.274600,2.161680


Checkpoint destination directory ./drive/MyDrive/Model_SparkLab/t5-small-finetuned/checkpoint-1000 already exists and is non-empty. Saving will proceed but saved results may be invalid.


In [ ]:
model.save_pretrained("./drive/MyDrive/Model_SparkLab/save_10epochs", from_pt=True)


In [ ]:
model_loaded = T5ForConditionalGeneration.from_pretrained("./drive/MyDrive/Model_SparkLab/save_manually")


In [ ]:
model_loaded.eval()
# Perform inference on the first sample in the eval set
first_eval_sample = eval_dataset[0]
print(first_eval_sample)
# Prepare the input for inference
input_ids = first_eval_sample['input_ids'].unsqueeze(0).to(model_loaded.device) # Add batch dimension
attention_mask = first_eval_sample['attention_mask'].unsqueeze(0).to(model_loaded.device)  # Add batch dimension

# Generate output using the model
output_ids = model_loaded.generate(input_ids=input_ids, attention_mask=attention_mask,
                            max_length=256)
# print(output_ids)
# Decode the output tokens
output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

print("Generated Output:", output_text)

{'input_ids': tensor([7255,   19,  251,  ...,    0,    0,    0]), 'attention_mask': tensor([1, 1, 1,  ..., 0, 0, 0]), 'labels': tensor([14011,   566, 20492,   348, 19844, 14011,   134,   440,    51,  1208,
        19844,  7187,    26,  8383,   771,    28,  2980,    16, 18456,   758,
            6,  3490,  5836,     6,    11,  1252,  8383,  2673,     5,   749,
         1926,  1418,    12,  2174,  6280,  3490,   251,    28, 23979,    11,
        12962,     5, 14011,  5420,  4267,  1433, 19844, 14011, 31190,  8383,
        18420, 19844,     6, 10793,     8,  1150,     3,    18,  3699,     6,
          784,  7681,    17,  7678,   908,     3,    18,   784,  8532,    26,
         7678,   908,  1429,     3, 28060,    38,     8,  2329,   500,    13,
          574,    21,  8383,    18,  3897, 20182,    11,   937,   380,    12,
         1652,    11,  3066,  7701,  1429,   461,  1976,    15,    26,   126,
         3804,     7,    11,  6997,  3490,  3187,  1429, 10272,    15,    26,
        18456,

In [ ]:
import re


In [ ]:
model.eval()
# Perform inference on the first sample in the eval set
first_eval_sample = eval_dataset[0]
print(first_eval_sample)
# Prepare the input for inference
input_ids = first_eval_sample['input_ids'].unsqueeze(0).to(model.device) # Add batch dimension
attention_mask = first_eval_sample['attention_mask'].unsqueeze(0).to(model.device)  # Add batch dimension

# Generate output using the model
output_ids = model.generate(input_ids=input_ids, attention_mask=attention_mask,
                            max_length=256)
# print(output_ids)
# Decode the output tokens
output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

print("Generated Output:", output_text)


{'input_ids': tensor([7255,   19,  251,  ...,    0,    0,    0]), 'attention_mask': tensor([1, 1, 1,  ..., 0, 0, 0]), 'labels': tensor([14011,   566, 20492,   348, 19844, 14011,   134,   440,    51,  1208,
        19844,  7187,    26,  8383,   771,    28,  2980,    16, 18456,   758,
            6,  3490,  5836,     6,    11,  1252,  8383,  2673,     5,   749,
         1926,  1418,    12,  2174,  6280,  3490,   251,    28, 23979,    11,
        12962,     5, 14011,  5420,  4267,  1433, 19844, 14011, 31190,  8383,
        18420, 19844,     6, 10793,     8,  1150,     3,    18,  3699,     6,
          784,  7681,    17,  7678,   908,     3,    18,   784,  8532,    26,
         7678,   908,  1429,     3, 28060,    38,     8,  2329,   500,    13,
          574,    21,  8383,    18,  3897, 20182,    11,   937,   380,    12,
         1652,    11,  3066,  7701,  1429,   461,  1976,    15,    26,   126,
         3804,     7,    11,  6997,  3490,  3187,  1429, 10272,    15,    26,
        18456,

In [ ]:
def remove_repeated_patterns(output_ids):
    # Convert the tensor to a list for easier manipulation
    output_list = output_ids.squeeze().tolist()

    def remove_repeats(lst):
        i = 0
        while i < len(lst):
            repeat_found = False
            # Look for repeating patterns starting from each position in the list
            for j in range(i + 1, len(lst)):
                if lst[i:j] == lst[j:j + (j - i)]:
                    # Remove the repeating sequence
                    lst = lst[:j] + lst[j + (j - i):]
                    repeat_found = True
                    break
            if not repeat_found:
                i += 1
        return lst

    # Remove repeating patterns from the output list
    cleaned_output_list = remove_repeats(output_list)

    # Convert the cleaned list back to a tensor
    cleaned_output_ids = torch.tensor(cleaned_output_list, device=output_ids.device).unsqueeze(0)
    return cleaned_output_ids

In [ ]:
model.eval()
# Perform inference on the first sample in the eval set
first_eval_sample = eval_dataset[4]
print(first_eval_sample)
# Prepare the input for inference
input_ids = first_eval_sample['input_ids'].unsqueeze(0).to(model.device) # Add batch dimension
attention_mask = first_eval_sample['attention_mask'].unsqueeze(0).to(model.device)  # Add batch dimension

# Generate output using the model
output_ids = model.generate(input_ids=input_ids, attention_mask=attention_mask,
                            max_length=256)
cleaned_ids = remove_repeated_patterns(output_ids)
# print(output_ids)
# Decode the output tokens
output_text = tokenizer.decode(cleaned_ids[0], skip_special_tokens=True)

print("Generated Output:", output_text)


{'input_ids': tensor([7255,   19,  251,  ..., 5589,   44,    1]), 'attention_mask': tensor([1, 1, 1,  ..., 1, 1, 1]), 'labels': tensor([14011,  2703,   189, 24042, 19844, 14011,   134,   440,    51,  1208,
        19844, 11281,    11,  2705,   771,    28,     3,     9,  3657,     5,
          308,     5,    16, 20383,  6923,    11,     3,     9,   446,     5,
          308,     5,  2937,    30,   807,    16,  6363,  1208,  6704,    11,
        27068,    12,  3165,  1614, 19788,     5,   749,  1926,  1418,    12,
          370,  1287,   884,   313,    11,  1865,  1561,   495,  7134,     7,
            5, 14011,  5420,  4267,  1433, 19844, 14011,   188,     7,     7,
          343,   288,  5589, 19844,     6,  1546,    26,  2897,   636,     6,
         1660,  1673,     3,    18, 18795,  1429,  2255,  1836,  2996,    16,
         4336,  4831,    11, 27068,  4831,  1429,  1980,  3466,    15,   481,
           30,  2705,    11,  1415,  1459,  1429, 21942,   585,    30,  6363,
         1208,

In [ ]:
def format_and_save_output(text, file_name="formatted_output.txt"):
    # Replace **{text}** with ||{text}|| and add a newline
    formatted_text = re.sub(r'\*\*(.*?)\*\*', r'\n\n||\1||', text)

    # Replace "* " with "\n- "
    formatted_text = formatted_text.replace("* ", "\n- ")

    # Save the formatted text to a file
    with open(file_name, "w") as file:
        file.write(formatted_text)

In [ ]:
format_and_save_output(output_text)

In [ ]:
def format_text(text, file_name="formatted_output.txt"):
    # Insert a newline after each period followed by a space
    text = text.replace(". ", ".\n")

    # Insert a newline before every '**' that is not at the beginning of a new line
    text = re.sub(r'(?<!\n)\*\*', r'\n**', text)

    # Insert a newline before each '*' that is not part of '**' and not at the beginning of a new line
    text = re.sub(r'(?<!\n)(?<!\*)\*(?!\*)', r'\n*', text)
    # Save the formatted text to a file
    with open(file_name, "w") as file:
        file.write(text)


In [ ]:
format_text(output_text)